In [ ]:
# Instalar dependencias
!pip install tensorflow
!pip install pandas matplotlib seaborn scikit-learn pillow

# Verificar GPU
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")
print(f"GPU activa: {tf.test.is_built_with_cuda()}")

2025-07-01 20:37:35.345876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751402255.547998      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751402255.610166      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow version: 2.18.0
GPU disponible: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
GPU activa: True


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import zipfile
import urllib.request
import shutil
import random
from PIL import Image

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB4, ResNet152
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

print("✅ Todas las dependencias importadas correctamente")


✅ Todas las dependencias importadas correctamente


In [3]:
import os
import shutil
import zipfile

def load_isic_dataset_from_kaggle():
    """Carga el dataset ISIC 2019 desde el dataset privado en Kaggle con estructura anidada"""
    print("🚀 Cargando dataset ISIC 2019 desde Kaggle...")

    base_path = 'dataset'

    # Ruta del CSV de etiquetas
    labels_path = os.path.join(base_path, 'ISIC_2019_Labels.csv')

    # Ruta de la carpeta que contiene las imágenes (ya descomprimidas)
    images_src_path = os.path.join(base_path, 'ISIC_2019_Training_Input')

    # Crear carpeta local para etiquetas e imágenes
    os.makedirs('data/ISIC_2019_Images', exist_ok=True)

    # Copiar etiquetas CSV a carpeta local
    if os.path.exists(labels_path):
        shutil.copy(labels_path, 'data/ISIC_2019_Labels.csv')
        print("📂 Etiquetas copiadas a data/ISIC_2019_Labels.csv")
    else:
        print(f"❌ No se encontró el archivo de etiquetas: {labels_path}")

    # Copiar imágenes a carpeta local (en lugar de extraer ZIP, ya están descomprimidas)
    if os.path.exists(images_src_path):
        print("📂 Copiando imágenes a carpeta local (esto puede tardar)...")
        # Copiar todos los archivos de imágenes
        for filename in os.listdir(images_src_path):
            src_file = os.path.join(images_src_path, filename)
            dst_file = os.path.join('data/ISIC_2019_Images', filename)
            shutil.copy(src_file, dst_file)
        print("✅ Imágenes copiadas exitosamente!")
    else:
        print(f"❌ No se encontró la carpeta de imágenes: {images_src_path}")

    return True

# Ejecutar función
load_isic_dataset_from_kaggle()


🚀 Cargando dataset ISIC 2019 desde Kaggle...
📂 Etiquetas copiadas a data/ISIC_2019_Labels.csv
📂 Copiando imágenes a carpeta local (esto puede tardar)...


KeyboardInterrupt: 

In [ ]:
import os
import shutil
import zipfile

def load_isic_dataset_from_kaggle():
    """Carga el dataset ISIC 2019 desde el dataset privado en Kaggle con estructura anidada"""
    print("🚀 Cargando dataset ISIC 2019 desde Kaggle...")

    base_path = 'dataset'

    # Ruta del CSV de etiquetas
    labels_path = os.path.join(base_path, 'ISIC_2019_Labels.csv')

    # Ruta de la carpeta que contiene las imágenes (ya descomprimidas)
    images_src_path = os.path.join(base_path, 'ISIC_2019_Training_Input', 'ISIC_2019_Training_Input')

    # Crear carpeta local para etiquetas e imágenes
    os.makedirs('data/ISIC_2019_Images', exist_ok=True)

    # Copiar etiquetas CSV a carpeta local
    if os.path.exists(labels_path):
        shutil.copy(labels_path, 'data/ISIC_2019_Labels.csv')
        print("📂 Etiquetas copiadas a data/ISIC_2019_Labels.csv")
    else:
        print(f"❌ No se encontró el archivo de etiquetas: {labels_path}")

    # Copiar imágenes a carpeta local (en lugar de extraer ZIP, ya están descomprimidas)
    if os.path.exists(images_src_path):
        print("📂 Copiando imágenes a carpeta local (esto puede tardar)...")
        # Copiar todos los archivos de imágenes
        for filename in os.listdir(images_src_path):
            src_file = os.path.join(images_src_path, filename)
            dst_file = os.path.join('data/ISIC_2019_Images', filename)
            shutil.copy(src_file, dst_file)
        print("✅ Imágenes copiadas exitosamente!")
    else:
        print(f"❌ No se encontró la carpeta de imágenes: {images_src_path}")

    return True

# Ejecutar función
load_isic_dataset_from_kaggle()


🚀 Cargando dataset ISIC 2019 desde Kaggle...
📂 Etiquetas copiadas a data/ISIC_2019_Labels.csv
📂 Copiando imágenes a carpeta local (esto puede tardar)...
✅ Imágenes copiadas exitosamente!


True

In [5]:
import os

image_dir = 'dataset/ISIC_2019_Images'
valid_extensions = ('.jpg', '.jpeg', '.png')

image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(valid_extensions)]
print(f"Número de imágenes en '{image_dir}': {len(image_files)}")


Número de imágenes en 'dataset/ISIC_2019_Images': 25331


In [9]:
def organize_isic_data():
    """Organiza el dataset ISIC en estructura train/test en Kaggle"""
    print("📁 Organizando dataset...")

    # Verifica si las imágenes existen
    if not Path('dataset/ISIC_2019_Images').exists():
        raise FileNotFoundError("❌ No se encontró la carpeta con las imágenes. Asegúrate de haber ejecutado la carga correctamente.")

    # Leer etiquetas
    labels_df = pd.read_csv('dataset/ISIC_2019_Labels.csv')
    print(f"📊 Total de imágenes: {len(labels_df)}")

    # Crear estructura de directorios
    train_dir = Path('data/ISIC_dataset')
    test_dir = Path('data/ISIC_dataset_test')

    train_dir.mkdir(exist_ok=True)
    test_dir.mkdir(exist_ok=True)

    # Crear subdirectorios
    (train_dir / 'benign').mkdir(exist_ok=True)
    (train_dir / 'malignant').mkdir(exist_ok=True)
    (test_dir / 'benign').mkdir(exist_ok=True)
    (test_dir / 'malignant').mkdir(exist_ok=True)

    # Semilla
    random.seed(42)

    # Contadores
    train_benign = 0
    train_malignant = 0
    test_benign = 0
    test_malignant = 0

    # Procesar cada imagen
    for index, row in labels_df.iterrows():
        image_name = row['image']

        is_malignant = float(row.get('MEL', 0)) == 1.0 or float(row.get('BCC', 0)) == 1.0
        is_benign = (
            float(row.get('NV', 0)) == 1.0 or
            float(row.get('BKL', 0)) == 1.0 or
            float(row.get('AK', 0)) == 1.0 or
            float(row.get('VASC', 0)) == 1.0 or
            float(row.get('DF', 0)) == 1.0
        )

        if is_malignant:
            class_name = 'malignant'
        elif is_benign:
            class_name = 'benign'
        else:
            continue

        source_path = Path('dataset/ISIC_2019_Images') / f"{image_name}.jpg"

        if source_path.exists():
            # 80% entrenamiento, 20% validación
            if random.random() < 0.8:
                dest_path = train_dir / class_name / f"{image_name}.jpg"
                if class_name == 'benign':
                    train_benign += 1
                else:
                    train_malignant += 1
            else:
                dest_path = test_dir / class_name / f"{image_name}.jpg"
                if class_name == 'benign':
                    test_benign += 1
                else:
                    test_malignant += 1

            shutil.copy2(source_path, dest_path)

    print(f"✅ Dataset organizado:")
    print(f"   Train benign: {train_benign} imágenes")
    print(f"   Train malignant: {train_malignant} imágenes")
    print(f"   Test benign: {test_benign} imágenes")
    print(f"   Test malignant: {test_malignant} imágenes")

    # ❌ No borrar archivos al final, ya que se cargaron desde /kaggle/input
    # os.remove(...)
    # shutil.rmtree(...)

    return True

# Ejecutar
organize_isic_data()


📁 Organizando dataset...
📊 Total de imágenes: 25331
✅ Dataset organizado:
   Train benign: 13497 imágenes
   Train malignant: 6300 imágenes
   Test benign: 3361 imágenes
   Test malignant: 1545 imágenes


True

In [6]:
import shutil

# Elimina las imágenes copiadas (originales)
shutil.rmtree('data/ISIC_2019_Images')
print("🗑️ Carpeta 'data/ISIC_2019_Images' eliminada para liberar espacio.")


🗑️ Carpeta 'data/ISIC_2019_Images' eliminada para liberar espacio.


In [7]:
def create_models():
    """Crea los 3 modelos para entrenamiento"""
    models = {}
    
    # Modelo 1: EfficientNetB4
    print("🔧 Creando modelo EfficientNetB4...")
    base_model = EfficientNetB4(
        weights='imagenet',
        include_top=False,
        input_shape=(300, 300, 3)
    )
    base_model.trainable = True
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    efficientnet = Model(inputs=base_model.input, outputs=predictions)
    efficientnet.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    models['EfficientNetB4'] = efficientnet
    
    # Modelo 2: ResNet152
    print("🚀 Creando modelo ResNet152...")
    base_model = ResNet152(
        weights='imagenet',
        include_top=False,
        input_shape=(300, 300, 3)
    )
    base_model.trainable = True
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    resnet = Model(inputs=base_model.input, outputs=predictions)
    resnet.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    models['ResNet152'] = resnet
    
    # Modelo 3: CNN Personalizada
    print("🚀 Creando modelo CNN Personalizada...")
    cnn_personalizada = tf.keras.Sequential([
        tf.keras.Input(shape=(300, 300, 3)),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    cnn_personalizada.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    models['CNN Personalizada'] = cnn_personalizada
    
    print("✅ Modelos creados exitosamente!")
    return models

# Crear modelos
models = create_models()

🔧 Creando modelo EfficientNetB4...


I0000 00:00:1751402764.088294      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751402764.090085      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
🚀 Creando modelo ResNet152...
234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
🚀 Creando modelo CNN Personalizada...
✅ Modelos creados exitosamente!


In [8]:
def prepare_data():
    """Prepara los datos para entrenamiento"""
    print("📊 Preparando datos...")
    
    # Data augmentation para entrenamiento
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    # Solo rescaling para validación
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    # Cargar datos
    train_generator = train_datagen.flow_from_directory(
        'data/ISIC_dataset',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary'
    )
    
    val_generator = val_datagen.flow_from_directory(
        'data/ISIC_dataset_test',
        target_size=(300, 300),
        batch_size=32,
        class_mode='binary'
    )
    
    print(f"✅ Datos preparados:")
    print(f"   Train: {train_generator.samples} muestras")
    print(f"   Validation: {val_generator.samples} muestras")
    print(f"   Batch size: 32")
    
    return train_generator, val_generator

# Preparar datos
train_gen, val_gen = prepare_data()

📊 Preparando datos...
Found 19797 images belonging to 2 classes.
Found 4906 images belonging to 2 classes.
✅ Datos preparados:
   Train: 19797 muestras
   Validation: 4906 muestras
   Batch size: 32


In [9]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Calcular class weights a partir de los datos de entrenamiento
labels = train_gen.classes  # 0 = benign, 1 = malignant
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights = dict(enumerate(class_weights_array))
print(f"📊 Class weights: {class_weights}")

📊 Class weights: {0: 0.73338519671038, 1: 1.5711904761904762}


In [10]:
import os
import zipfile
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

def train_and_export_single_model(model_name, models, train_gen, val_gen, class_weights):
    """Entrena un único modelo y lo exporta comprimido en ZIP"""
    print(f"\n{'='*60}")
    print(f"🚀 Entrenando modelo: {model_name}")
    print(f"{'='*60}")

    model = models[model_name]
    
    # Epochs por tipo de modelo
    epochs = 30 if "CNN Personalizada" in model_name else 20

    # Asegurar carpeta de modelos
    os.makedirs('models', exist_ok=True)

    # Callbacks
    callbacks = [
        ModelCheckpoint(
            filepath=f'models/{model_name.lower().replace(" ", "_")}.h5',
            save_best_only=True,
            monitor='val_accuracy',
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]

    # Entrenamiento
    history = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // train_gen.batch_size,
        validation_data=val_gen,
        validation_steps=val_gen.samples // val_gen.batch_size,
        epochs=epochs,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1
    )

    # Evaluación
    val_loss, val_acc, val_prec, val_rec = model.evaluate(val_gen, verbose=0)
    print(f"\n✅ Resultados finales de {model_name}:")
    print(f"   Accuracy: {val_acc:.4f}")
    print(f"   Precision: {val_prec:.4f}")
    print(f"   Recall: {val_rec:.4f}")

    # Comprimir modelo
    model_file = f'models/{model_name.lower().replace(" ", "_")}.h5'
    zip_file = f'{model_name.lower().replace(" ", "_")}.zip'
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(model_file)

    print(f"📦 Modelo comprimido en: {zip_file}")
    
train_and_export_single_model('EfficientNetB4', models, train_gen, val_gen, class_weights)


🚀 Entrenando modelo: EfficientNetB4


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1751403012.150780     110 service.cc:148] XLA service 0x7ba7e4004270 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751403012.152019     110 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751403012.152043     110 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751403022.382762     110 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1751403046.131288     110 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751403046.283388     110 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751403046.642930     110 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. Th

240/618 ━━━━━━━━━━━━━━━━━━━━ 5:52 932ms/step - accuracy: 0.6606 - loss: 0.6389 - precision: 0.4778 - recall: 0.6611

E0000 00:00:1751403340.028059     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751403340.172455     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751403340.558581     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751403340.702759     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1751403341.093235     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6892 - loss: 0.5866 - precision: 0.5108 - recall: 0.7035
Epoch 1: val_accuracy improved from -inf to 0.68546, saving model to models/efficientnetb4.h5
618/618 ━━━━━━━━━━━━━━━━━━━━ 894s 1s/step - accuracy: 0.6892 - loss: 0.5865 - precision: 0.5108 - recall: 0.7036 - val_accuracy: 0.6855 - val_loss: 0.6092 - val_precision: 0.7500 - val_recall: 0.0039 - learning_rate: 0.0010
Epoch 2/20
  1/618 ━━━━━━━━━━━━━━━━━━━━ 7:56 773ms/step - accuracy: 0.7812 - loss: 0.5703 - precision: 0.5714 - recall: 0.8889

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy improved from 0.68546 to 0.68566, saving model to models/efficientnetb4.h5
618/618 ━━━━━━━━━━━━━━━━━━━━ 53s 84ms/step - accuracy: 0.7812 - loss: 0.5703 - precision: 0.5714 - recall: 0.8889 - val_accuracy: 0.6857 - val_loss: 0.6114 - val_precision: 1.0000 - val_recall: 0.0019 - learning_rate: 0.0010
Epoch 3/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 866ms/step - accuracy: 0.7711 - loss: 0.4805 - precision: 0.6109 - recall: 0.7753
Epoch 3: val_accuracy did not improve from 0.68566
618/618 ━━━━━━━━━━━━━━━━━━━━ 587s 949ms/step - accuracy: 0.7711 - loss: 0.4805 - precision: 0.6109 - recall: 0.7753 - val_accuracy: 0.6846 - val_loss: 0.8072 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 4/20
  1/618 ━━━━━━━━━━━━━━━━━━━━ 7:36 740ms/step - accuracy: 0.9062 - loss: 0.2931 - precision: 0.9000 - recall: 0.8182
Epoch 4: val_accuracy did not improve from 0.68566

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
618/618 ━━

In [11]:
import os
import zipfile
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

def train_and_export_single_model(model_name, models, train_gen, val_gen, class_weights):
    """Entrena un único modelo y lo exporta comprimido en ZIP"""
    print(f"\n{'='*60}")
    print(f"🚀 Entrenando modelo: {model_name}")
    print(f"{'='*60}")

    model = models[model_name]
    
    # Epochs por tipo de modelo
    epochs = 30 if "CNN Personalizada" in model_name else 20

    # Asegurar carpeta de modelos
    os.makedirs('models', exist_ok=True)

    # Callbacks
    callbacks = [
        ModelCheckpoint(
            filepath=f'models/{model_name.lower().replace(" ", "_")}.h5',
            save_best_only=True,
            monitor='val_accuracy',
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]

    # Entrenamiento
    history = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // train_gen.batch_size,
        validation_data=val_gen,
        validation_steps=val_gen.samples // val_gen.batch_size,
        epochs=epochs,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1
    )

    # Evaluación
    val_loss, val_acc, val_prec, val_rec = model.evaluate(val_gen, verbose=0)
    print(f"\n✅ Resultados finales de {model_name}:")
    print(f"   Accuracy: {val_acc:.4f}")
    print(f"   Precision: {val_prec:.4f}")
    print(f"   Recall: {val_rec:.4f}")

    # Comprimir modelo
    model_file = f'models/{model_name.lower().replace(" ", "_")}.h5'
    zip_file = f'{model_name.lower().replace(" ", "_")}.zip'
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(model_file)

    print(f"📦 Modelo comprimido en: {zip_file}")
    
train_and_export_single_model('ResNet152', models, train_gen, val_gen, class_weights)


🚀 Entrenando modelo: ResNet152
Epoch 1/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5748 - loss: 0.7266 - precision: 0.3945 - recall: 0.6675
Epoch 1: val_accuracy improved from -inf to 0.68505, saving model to models/resnet152.h5
618/618 ━━━━━━━━━━━━━━━━━━━━ 1362s 2s/step - accuracy: 0.5748 - loss: 0.7264 - precision: 0.3945 - recall: 0.6676 - val_accuracy: 0.6850 - val_loss: 0.6019 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/20
  1/618 ━━━━━━━━━━━━━━━━━━━━ 15:47 2s/step - accuracy: 0.6562 - loss: 0.6001 - precision: 0.5000 - recall: 0.8182
Epoch 2: val_accuracy did not improve from 0.68505
618/618 ━━━━━━━━━━━━━━━━━━━━ 79s 126ms/step - accuracy: 0.6562 - loss: 0.6001 - precision: 0.5000 - recall: 0.8182 - val_accuracy: 0.6840 - val_loss: 0.6080 - val_precision: 0.2000 - val_recall: 6.4767e-04 - learning_rate: 0.0010
Epoch 3/20
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6006 - loss: 0.6287 - precision: 0.4238 - recall: 0.7

In [ ]:
import os
import zipfile
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

def train_and_export_single_model(model_name, models, train_gen, val_gen, class_weights):
    """Entrena un único modelo y lo exporta comprimido en ZIP"""
    print(f"\n{'='*60}")
    print(f"🚀 Entrenando modelo: {model_name}")
    print(f"{'='*60}")

    model = models[model_name]
    
    # Epochs por tipo de modelo
    epochs = 30 if "CNN Personalizada" in model_name else 20

    # Asegurar carpeta de modelos
    os.makedirs('models', exist_ok=True)

    # Callbacks
    callbacks = [
        ModelCheckpoint(
            filepath=f'models/{model_name.lower().replace(" ", "_")}.h5',
            save_best_only=True,
            monitor='val_accuracy',
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]

    # Entrenamiento
    history = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // train_gen.batch_size,
        validation_data=val_gen,
        validation_steps=val_gen.samples // val_gen.batch_size,
        epochs=epochs,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1
    )

    # Evaluación
    val_loss, val_acc, val_prec, val_rec = model.evaluate(val_gen, verbose=0)
    print(f"\n✅ Resultados finales de {model_name}:")
    print(f"   Accuracy: {val_acc:.4f}")
    print(f"   Precision: {val_prec:.4f}")
    print(f"   Recall: {val_rec:.4f}")

    # Comprimir modelo
    model_file = f'models/{model_name.lower().replace(" ", "_")}.h5'
    zip_file = f'{model_name.lower().replace(" ", "_")}.zip'
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(model_file)

    print(f"📦 Modelo comprimido en: {zip_file}")
    
train_and_export_single_model('CNN Personalizada', models, train_gen, val_gen, class_weights)